In [1]:
from pc import PS
from modules import ADC,DAC,CHIP
from cimCommand import CMD,CmdData,Packet
from cimCommand.singleCmdInfo import *
import matplotlib.pyplot as plt
import numpy as np

import time
import json

In [2]:
chip=CHIP(PS("192.168.1.10", 7,delay=0.3,debug=False))

Failed to connect: [WinError 10049] 在其上下文中，该请求的地址无效。


In [3]:
chip.set_device_cfg(deviceType=1)

In [4]:
chip.ps.debug = True
chip.set_op_mode2(read=True,row=False)
vres,_ = chip.read2(row_index=[i for i in range(256)],col_index=[0],read_voltage=0.1,tia_flag=True)
print(vres)

操作模式：read	器件：ECRAM	读电压：0.1v	从行\列给电压：列	TIA增益：0
模式：5
	帧头:               	字节码: 55 aa 05
	指令: pl_ram_addr   	字节码: 00 00
	指令: pl_data_length	字节码: 00 81
	指令: pl_data       	字节码: 00 00 00 01
	指令: pl_data       	字节码: 80 00 80 00
	指令: pl_data       	字节码: 80 00 80 00
	指令: pl_data       	字节码: 80 00 80 00
	指令: pl_data       	字节码: 80 00 80 00
	指令: pl_data       	字节码: 80 00 80 00
	指令: pl_data       	字节码: 80 00 80 00
	指令: pl_data       	字节码: 80 00 80 00
	指令: pl_data       	字节码: 80 00 80 00
	指令: pl_data       	字节码: 40 00 40 00
	指令: pl_data       	字节码: 40 00 40 00
	指令: pl_data       	字节码: 40 00 40 00
	指令: pl_data       	字节码: 40 00 40 00
	指令: pl_data       	字节码: 40 00 40 00
	指令: pl_data       	字节码: 40 00 40 00
	指令: pl_data       	字节码: 40 00 40 00
	指令: pl_data       	字节码: 40 00 40 00
	指令: pl_data       	字节码: 20 00 20 00
	指令: pl_data       	字节码: 20 00 20 00
	指令: pl_data       	字节码: 20 00 20 00
	指令: pl_data       	字节码: 20 00 20 00
	指令: pl_data       	字节码: 20 00 20 00
	指令: pl_data       	字节码: 20 00 20 00
	指

AttributeError: 'NoneType' object has no attribute 'hex'

In [5]:
ps.debug = True

In [7]:
cmd=[
    CMD(PL_DATA,command_data=CmdData(0x1234_5678)),
    CMD(PL_DATA,command_data=CmdData(0x8765_4321)),
]

cmd.insert(0,CMD(PL_DATA_LENGTH,command_data=CmdData(len(cmd))))
cmd.insert(0,CMD(PL_RAM_ADDR,command_data=CmdData(0)))

pkts=Packet()
pkts.append_single(cmd,mode=5)
ps.send_packets(pkts)

模式：5,指令：read_din_ram,指令：pl_data_length,指令：pl_data,指令：pl_data
字节码：55 aa 05 00 00 00 02 12 34 56 78 87 65 43 21



In [4]:
ps.debug = True
ps.enable = True

In [5]:
cmd=[
    CMD(PL_ROW_BANK,command_data=CmdData(1<<(1+8) | 0)),
    CMD(PL_COL_BANK,command_data=CmdData(1<<(2+8) | 1)),
    CMD(PL_DAC_V,command_data=CmdData(12<<16 | 0xFFF0)),
    CMD(PL_WRITE_ROW_PULSE),
    CMD(PL_WRITE_COL_PULSE),
    CMD(PL_READ_ROW_PULSE,command_data=CmdData(0)),
    CMD(PL_READ_COL_PULSE,command_data=CmdData(1)),
]
num = len(cmd)
cmd.insert(0,CMD(PL_DATA_LENGTH,command_data=CmdData(num)))
cmd.insert(0,CMD(PL_RAM_ADDR,command_data=CmdData(0)))

pkts=Packet()
pkts.append_single(cmd,mode=4)
pkts.append_cmdlist([
    CMD(INS_NUM,command_data=CmdData(num)),
    CMD(FAST_COMMAND_1,command_data=CmdData(FAST_COMMAND1_CONF.cfg_ins_run)),
],mode=1)

ps.send_packets(pkts)
ps.send_packets(pkts)

模式：4	指令: pl_ram_addr       	字节码: 00 00
	指令: pl_data_length    	字节码: 00 07
	指令: pl_row_bank       	字节码: 01 00 02 00
	指令: pl_col_bank       	字节码: 02 00 04 01
	指令: pl_dac_v          	字节码: 03 0c ff f0
	指令: pl_write_row_pulse	字节码: 06 00 00 00
	指令: pl_write_col_pulse	字节码: 07 00 00 00
	指令: pl_read_row_pulse 	字节码: 04 00 00 00
	指令: pl_read_col_pulse 	字节码: 05 00 00 01
模式：1	指令: ins_num           	字节码: 1c 00 00 00 07
模式：1	指令: fast_command_1    	字节码: 01 00 00 80 00

Failed to send message:
模式：4	指令: pl_ram_addr       	字节码: 00 00
	指令: pl_data_length    	字节码: 00 07
	指令: pl_row_bank       	字节码: 01 00 02 00
	指令: pl_col_bank       	字节码: 02 00 04 01
	指令: pl_dac_v          	字节码: 03 0c ff f0
	指令: pl_write_row_pulse	字节码: 06 00 00 00
	指令: pl_write_col_pulse	字节码: 07 00 00 00
	指令: pl_read_row_pulse 	字节码: 04 00 00 00
	指令: pl_read_col_pulse 	字节码: 05 00 00 01
模式：1	指令: ins_num           	字节码: 1c 00 00 00 07
模式：1	指令: fast_command_1    	字节码: 01 00 00 80 00

Failed to send message:


In [11]:
cmd=[
    CMD(PL_RAM_ADDR,command_data=CmdData(0)),
    CMD(PL_DATA_LENGTH,command_data=CmdData(2))
    # CMD(PL_DAC_V,command_data=CmdData(dac_channel<<16 | voltage))
]

# cmd.insert(0,CMD(PL_DATA_LENGTH,command_data=CmdData(len(cmd))))
# cmd.insert(0,CMD(PL_RAM_ADDR,command_data=CmdData(0)))

pkts=Packet()
pkts.append_single(cmd,mode=6)
ps.send_packets(pkts)



message=ps.receive_packet()
print(message.hex())

模式：6,指令：read_dout_ram,指令：pl_data_length
字节码：55 aa 06 00 00 00 02

Received: b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc0\x00\x90\xff\xff\xff\xff\x00%\x00m\xff\xff\xff\xff\x00\x16\xff\xc7\xff\xff\xff\xff\x00[\x002\xff\xff\xff\xff'

000000000000000000000000000000000000000000000000000000000000000000c00090ffffffff0025006dffffffff0016ffc7ffffffff005b0032ffffffff


In [1]:

row_bank=[[],[1]]
row_bank = [i for i in row_bank if len(i)>0]
print(row_bank)

[[1]]


In [ ]:
cmd=[
    CMD(PL_DATA,command_data=CmdData(0x1234_5678)),
    CMD(PL_DATA,command_data=CmdData(0x8765_4321)),
    # CMD(PL_DATA,command_data=CmdData(0xFFFF_FFFF)),
]

cmd.insert(0,CMD(PL_DATA_LENGTH,command_data=CmdData(len(cmd))))
cmd.insert(0,CMD(PL_RAM_ADDR,command_data=CmdData(0)))

pkts=Packet()
pkts.append_single(cmd,mode=5)
ps.send_packets(pkts)

In [16]:
# chip.read_set(row=True,v=0.1,gain=3)
message_cond = []
message_v = []
for TIA_num in [i for i in range(16)]:
    adc_num, adc_channel = int(TIA_num/4),TIA_num%4

    channel_map = {0: 'A', 1: 'B', 2: 'C', 3: 'D'}
    pkts=Packet()
    adc_out = CMD(dict(
        command_addr = 65+TIA_num,
        command_type = COMMAND_TYPE[1],
        n_addr_bytes = N_ADDR_BYTES[0],
        n_data_bytes = N_DATA_BYTES[0],
        command_name = f"adc{adc_num}_out{channel_map[adc_channel]}",
        command_data = CmdData(0),
        command_description = "从ADC对应通道读取数据"
    ))
    pkts.append_cmdlist([adc_out],mode=2)
    print(pkts)
    # 发送指令
    ps.send_packets(pkts)
    # 接收信息
    # 高16bit：0，低16bit：寄存器的16bit值
    message = ps.receive_packet(f"adc读取:{adc_out.command_name}")
    print(message.hex())

模式：2,指令：adc0_outA
字节码：55 aa 02 41

b1000000
模式：2,指令：adc0_outB
字节码：55 aa 02 42

c7040000
模式：2,指令：adc0_outC
字节码：55 aa 02 43

ffffffff
模式：2,指令：adc0_outD
字节码：55 aa 02 44

ffffffff
模式：2,指令：adc1_outA
字节码：55 aa 02 45

b4000000
模式：2,指令：adc1_outB
字节码：55 aa 02 46

4a040000
模式：2,指令：adc1_outC
字节码：55 aa 02 47

ffffffff
模式：2,指令：adc1_outD
字节码：55 aa 02 48

ffffffff
模式：2,指令：adc2_outA
字节码：55 aa 02 49

dd030000
模式：2,指令：adc2_outB
字节码：55 aa 02 4a

3a000000
模式：2,指令：adc2_outC
字节码：55 aa 02 4b

ffffffff
模式：2,指令：adc2_outD
字节码：55 aa 02 4c

ffffffff
模式：2,指令：adc3_outA
字节码：55 aa 02 4d

76000000
模式：2,指令：adc3_outB
字节码：55 aa 02 4e

68040000
模式：2,指令：adc3_outC
字节码：55 aa 02 4f

ffffffff
模式：2,指令：adc3_outD
字节码：55 aa 02 50

ffffffff


In [17]:
# bank = 1
# din_ram_add = 10

# dac_channel = 12

cmd=[
    CMD(PL_RAM_ADDR,command_data=CmdData(0)),
    CMD(PL_DATA_LENGTH,command_data=CmdData(1))
    # CMD(PL_DAC_V,command_data=CmdData(dac_channel<<16 | voltage))
]

# cmd.insert(0,CMD(PL_DATA_LENGTH,command_data=CmdData(len(cmd))))
# cmd.insert(0,CMD(PL_RAM_ADDR,command_data=CmdData(0)))

pkts=Packet()
pkts.append_single(cmd,mode=6)
ps.send_packets(pkts)
print(pkts)


message=ps.receive_packet()
print(message.hex())

模式：6,指令：read_dout_ram,指令：pl_data_length
字节码：55 aa 06 00 00 00 01

04c700b1ffffffff044a00b4ffffffff003a03ddffffffff04680076ffffffff


In [6]:
print(message.hex()[0:4])
# adc通道 1，0，3，2...... 

04c3


In [9]:
test=CMD(DAC_IN,command_data=CmdData(1<<24|0x0400ff))

test = CMD(CIM_BANK_SEL,command_data=CmdData(1<<7))
print(bytes(test))


print(test.concatenate().to_bytes((test.n_addr_bytes + test.n_data_bytes), "little"))

b'\n\x00\x00\x00\x80'
b'\x80\x00\x00\x00\n'


In [11]:
value = 0x1111_FFFF
big_endian_bytes = value.to_bytes(4, byteorder='big')

print(big_endian_bytes)

b'\x11\x11\xff\xff'


In [2]:
# 用于和下位机交互通信
ps=PS(1,1,delay=10*1e-3,debug=False)
# 控制ADC
adc=ADC(ps)
# 控制DAC
dac=DAC(ps)
# 芯片，读写操作，device_cfg
chip=CHIP(adc,dac,ps)

Failed to connect: [WinError 10049] 在其上下文中，该请求的地址无效。


In [3]:
chip.set_device_cfg(deviceType=1)
chip.set_read_mode(row=True)
chip.set_dac_read_V(0.1)
chip.adc.set_gain(3)
# i,num,bank，index，tia
res=chip.read_row_or_col(row_index=[1],col_index=[2,54,19,10,120,240,197,127,138])
print(res)

操作模式：read	器件：ECRAM	读电压：0.1v	从行\列给电压：行	TIA增益：3
映射bank [[10, 54], [120], [138], [240], [19], [127], [], [197]]


AttributeError: 'NoneType' object has no attribute 'hex'

In [4]:
for i in range(0,255,2):
    print(i+1,"\t",chip.numToBank_Index(i))

for i in range(1,256,2):
    print(i+1,"\t",chip.numToBank_Index(i))

1 	 (0, 0)
3 	 (0, 1)
5 	 (0, 2)
7 	 (0, 3)
9 	 (0, 4)
11 	 (0, 5)
13 	 (0, 6)
15 	 (0, 7)
17 	 (0, 8)
19 	 (0, 9)
21 	 (0, 10)
23 	 (0, 11)
25 	 (0, 12)
27 	 (0, 13)
29 	 (0, 14)
31 	 (0, 15)
33 	 (0, 16)
35 	 (0, 17)
37 	 (0, 18)
39 	 (0, 19)
41 	 (0, 20)
43 	 (0, 21)
45 	 (0, 22)
47 	 (0, 23)
49 	 (0, 24)
51 	 (0, 25)
53 	 (0, 26)
55 	 (0, 27)
57 	 (0, 28)
59 	 (0, 29)
61 	 (0, 30)
63 	 (0, 31)
65 	 (1, 0)
67 	 (1, 1)
69 	 (1, 2)
71 	 (1, 3)
73 	 (1, 4)
75 	 (1, 5)
77 	 (1, 6)
79 	 (1, 7)
81 	 (1, 8)
83 	 (1, 9)
85 	 (1, 10)
87 	 (1, 11)
89 	 (1, 12)
91 	 (1, 13)
93 	 (1, 14)
95 	 (1, 15)
97 	 (1, 16)
99 	 (1, 17)
101 	 (1, 18)
103 	 (1, 19)
105 	 (1, 20)
107 	 (1, 21)
109 	 (1, 22)
111 	 (1, 23)
113 	 (1, 24)
115 	 (1, 25)
117 	 (1, 26)
119 	 (1, 27)
121 	 (1, 28)
123 	 (1, 29)
125 	 (1, 30)
127 	 (1, 31)
129 	 (2, 0)
131 	 (2, 1)
133 	 (2, 2)
135 	 (2, 3)
137 	 (2, 4)
139 	 (2, 5)
141 	 (2, 6)
143 	 (2, 7)
145 	 (2, 8)
147 	 (2, 9)
149 	 (2, 10)
151 	 (2, 11)
153 	 (2, 12)
155 	 (

In [5]:
for i in range(0,255,2):
    print(i+1,"\t",adc.TIA_index_map(i,device=0,col=True))

for i in range(1,256,2):
    print(i+1,"\t",adc.TIA_index_map(i,device=0,col=True))

1 	 8
3 	 8
5 	 8
7 	 8
9 	 8
11 	 8
13 	 8
15 	 8
17 	 8
19 	 8
21 	 8
23 	 8
25 	 8
27 	 8
29 	 8
31 	 8
33 	 9
35 	 9
37 	 9
39 	 9
41 	 9
43 	 9
45 	 9
47 	 9
49 	 9
51 	 9
53 	 9
55 	 9
57 	 9
59 	 9
61 	 9
63 	 9
65 	 10
67 	 10
69 	 10
71 	 10
73 	 10
75 	 10
77 	 10
79 	 10
81 	 10
83 	 10
85 	 10
87 	 10
89 	 10
91 	 10
93 	 10
95 	 10
97 	 11
99 	 11
101 	 11
103 	 11
105 	 11
107 	 11
109 	 11
111 	 11
113 	 11
115 	 11
117 	 11
119 	 11
121 	 11
123 	 11
125 	 11
127 	 11
129 	 12
131 	 12
133 	 12
135 	 12
137 	 12
139 	 12
141 	 12
143 	 12
145 	 12
147 	 12
149 	 12
151 	 12
153 	 12
155 	 12
157 	 12
159 	 12
161 	 13
163 	 13
165 	 13
167 	 13
169 	 13
171 	 13
173 	 13
175 	 13
177 	 13
179 	 13
181 	 13
183 	 13
185 	 13
187 	 13
189 	 13
191 	 13
193 	 14
195 	 14
197 	 14
199 	 14
201 	 14
203 	 14
205 	 14
207 	 14
209 	 14
211 	 14
213 	 14
215 	 14
217 	 14
219 	 14
221 	 14
223 	 14
225 	 15
227 	 15
229 	 15
231 	 15
233 	 15
235 	 15
237 	 15
239 	 15
241 	 1

In [9]:
print(adc.TIA_index_map(33))

9


### 测试indexMap

In [4]:
x = 255
bank,index = numToBank_Index(x)
print(f"bank = {bank}, index = {index}")

bank = 3, index = 31


In [5]:
import cimCommand.singleCmdInfo as SCD

singleCmd=[]
for attribute_name in dir(SCD):
    attribute_value = getattr(SCD, attribute_name)
    if type(attribute_value)==dict:
        command_name=attribute_value.get("command_name",None)
        if command_name:
            singleCmd.append(attribute_value)
singleCmd.sort(key=lambda x: x['command_addr'])
for k,v in enumerate(singleCmd):
    # print(v)
    print(CMD(v))

地址:	0
类型:	RW
位宽:	32 bit
名字:	fast_command_0
数据:	0
字节:	b'\x00\x00\x00\x00\x00'
描述:	触发PL运行某功能
        bit位   功能
        14	cfg_col_read	产生col read pulse脉冲，adc求平均返回寄存器
        13	cfg_row_read	产生row read pulse脉冲，adc求平均返回寄存器
        12	cfg_col_pulse	
        11	cfg_row_pulse	
        10	cfg_latch_clk	
        9	cfg_reg_clk	
        8	cfg_flt_le2	
        7	cfg_flt_le1	
        6	cfg_bank_sel	
        5	cfg_cim_data_in	配置32bit data
        4	cfg_adc3	配置ADC3
        3	cfg_adc2	配置ADC2
        2	cfg_adc1	配置ADC1
        1	cfg_adc0	配置ADC0
        0	cfg_dac	配置DAC

地址:	1
类型:	RW
位宽:	32 bit
名字:	fast_command_1
数据:	0
字节:	b'\x00\x00\x00\x00\x01'
描述:	触发PL运行某功能

地址:	2
类型:	RW
位宽:	32 bit
名字:	dac_in
数据:	0
字节:	b'\x00\x00\x00\x00\x02'
描述:	高8bit：0或1，选择dac。低24bit：spi写入DAC寄存器的24bit值

地址:	3
类型:	RW
位宽:	32 bit
名字:	adc0_in
数据:	0
字节:	b'\x00\x00\x00\x00\x03'
描述:	高16bit：0，低16bit，spi写入ADC寄存器的16bit值

地址:	4
类型:	RW
位宽:	32 bit
名字:	adc1_in
数据:	0
字节:	b'\x00\x00\x00\x00\x04'
描述:	高16bit：0，低16bit，spi写入ADC寄存器的16bit值

地址:	5
类型:	RW
位

In [6]:
from CIMcommand import CMD,FAST_COMMAND_0,DAC_IN,DAC_CTR
from util import *
print(CMD(DAC_IN,command_data=(1<<24)|7))
print(CMD(DAC_CTR))

ModuleNotFoundError: No module named 'CIMcommand'

In [4]:
data=0xF
data=set_zero(data,5)
print(bin(data))

data=0xF
data=set_one(data,5)
print(bin(data))

data=0xFFFF
data=place_bit(data,1,4,0b0000)
print(bin(data))

0b1111
0b101111
0b1111111111111111 0b11110 0b0
0b1111111111100001


In [ ]:
from pc.commands import *
from pc.packets import *
from pc.ps import *

: 

In [3]:
cmd_list = [DAC_IN(0, 10), FAST_COMMAND_1(1)]
packet = Packet(0, cmd_list)
print(bytes(cmd_list[0]))
# print(bytes(packet))

# split_list = [1, 2, 3, 7, 8, 12]
# split_name_list = ['帧头', '模式', 'reg地址', 'reg值', 'cmd地址', 'cmd值']
# split_index = 0
# res_bin = ""
# for index, byte in enumerate(bytes(packet)):
#     res_bin += format(byte, '08b')
#     res_bin += "\t"
#     if index == split_list[split_index]:
#         print(split_name_list[split_index] + "\t\t" + res_bin)
#         res_bin = ""
#         split_index += 1

b'\n\x00\x00\x00\x02'


In [3]:
ps = PS("192.168.1.10", 7)


Connected to 192.168.1.10:7
